In [5]:
import foolbox as fb
import art
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import joblib

In [6]:
X_train = joblib.load('../../preprocessed_data/CICIDS/X_train.joblib')
X_test = joblib.load('../../preprocessed_data/CICIDS/X_test.joblib')
Y_train = joblib.load('../../preprocessed_data/CICIDS/Y_train.joblib')
Y_test = joblib.load('../../preprocessed_data/CICIDS/Y_test.joblib')

In [7]:
from tensorflow.keras.models import load_model

cnn_loaded = load_model('../../models/cnn_CICIDS2017_model.keras')
lstm_loaded = load_model('../../models/lstm_CICIDS2017_model.keras')
ae_lstm_loaded = load_model('../../models/ae-lstm_CICIDS2017_model.keras')

2025-06-30 10:04:23.040486: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-06-30 10:04:23.056482: W tensorflow/c/c_api.cc:304] Operation '{name:'conv1d_2/bias/Assign' id:20 op device:{requested: '', assigned: ''} def:{{{node conv1d_2/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv1d_2/bias, conv1d_2/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2025-06-30 10:04:23.138814: W tensorflow/c/c_api.cc:304] Operation '{name:'total/Assign' id:134 op device:{requested: '', assigned: ''} def:{{{node total/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](total, total/Initializer/zeros)}}' was changed by setting attribute after

### FGSM

In [8]:
# from art.attacks.evasion import FastGradientMethod
# from art.estimators.classification import KerasClassifier

# # attack_fgsm = fgsmAttack(estimator=classifier)

# classifier = KerasClassifier(model=cnn_loaded)

# attack_fgsm = FastGradientMethod(estimator=classifier)

# adv_fgsm = attack_fgsm.generate(x=X_test[:2000], y=Y_test[:2000])

# # fgsm predictions
# preds_adv_fgsm = classifier.predict(adv_fgsm)
# labels_adv_fgsm = (preds_adv_fgsm > 0.5).astype(int).flatten()

# print("=== Prediction Counts ===")
# print("Original        :", Counter(labels_orig))
# print("fgsmAttack  :", Counter(labels_adv_fgsm))

from collections import Counter
from art.attacks.evasion import FastGradientMethod, ProjectedGradientDescent
from art.estimators.classification import KerasClassifier

# attack_fgsm = fgsmAttack(estimator=classifier)
classifier = KerasClassifier(model=cnn_loaded, clip_values=(X_test.min(), X_test.max()))
classifier._model.trainable = False

preds_clean = classifier.predict(X_test[:2000])
labels_clean = (preds_clean > 0.5).astype(int).flatten()
print(X_test.min(), X_test.max())


# only keep correctly classified traffic (TP and TN)
correct = labels_clean == Y_test[:2000]

X_correct = X_test[:2000][correct]
Y_correct = Y_test[:2000][correct]

attack_fgsm = FastGradientMethod(estimator=classifier, eps=1.0)
adv_fgsm = attack_fgsm.generate(x=X_correct, y=Y_correct)

attack_pgd = ProjectedGradientDescent(estimator=classifier, eps=1.0)
adv_pgd = attack_pgd.generate(x=X_correct, y=Y_correct)

#original predictions
preds_orig = classifier.predict(X_correct)
labels_orig = (preds_orig > 0.5).astype(int).flatten()

# fgsm predictions
preds_adv_fgsm = classifier.predict(adv_fgsm)
labels_adv_fgsm = (preds_adv_fgsm > 0.5).astype(int).flatten()

# ProjectedGradientDescent predictions
preds_adv_pgd = classifier.predict(adv_pgd)
# print(preds_orig[10:])
# print(preds_adv_pgd[10:])
labels_adv_pgd = (preds_adv_pgd > 0.5).astype(int).flatten()

print("=== Prediction Counts ===")
print("Original        :", Counter(labels_orig))
print("fgsmAttack  :", Counter(labels_adv_fgsm))
print("ProjectedGradientDescent  :", Counter(labels_adv_pgd))

/home/bossantm/anaconda3/envs/SDNIOT-env/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2025-06-30 10:04:24.265612: W tensorflow/c/c_api.cc:304] Operation '{name:'dense_10/Sigmoid' id:105 op device:{requested: '', assigned: ''} def:{{{node dense_10/Sigmoid}} = Sigmoid[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_10/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


-620.8767 536.92633


PGD - Random Initializations: 100%|██████████| 1/1 [00:00<00:00,  6.76it/s]


=== Prediction Counts ===
Original        : Counter({1: 983, 0: 968})
fgsmAttack  : Counter({0: 1459, 1: 492})
ProjectedGradientDescent  : Counter({1: 979, 0: 972})


### Trying Foolbox

In [ ]:
fmodel = fb.TensorFlowModel(cnn_loaded, bounds=(X_test.min(), X_test.max()))

# attack = fb.attacks.HopSkipJumpAttack()
attack = fb.attacks.BoundaryAttack()
# attack = fb.attacks.FGSM()


epsilons = [0.01, 0.05, 0.1, 0.2, 0.3, 0.6, 0.8, 1.0]
x = tf.constant(X_test[:5].astype(np.float32))
y = tf.constant(Y_test[:5].astype(np.int64))

# preds = tf.argmax(fmodel(x), axis=-1)
# mask = preds != y
# x_filtered = tf.boolean_mask(x, mask)
# y_filtered = tf.boolean_mask(y, mask)

# print(f"{len(x_filtered)} correctly classified inputs to attack")

# if len(x_filtered) == 0:
#     raise RuntimeError("No sample correctly classified to launch the attack")

advs, clipped, success = attack(fmodel, x, y, epsilons=epsilons)

for i, eps in enumerate(epsilons):
    print(f"Epsilon {eps:.2f} — Success rate: {np.mean(success[i]):.2%}")


/home/bossantm/anaconda3/envs/dl-attack-env/lib/python3.9/site-packages/foolbox/attacks/blended_noise.py:85: UserWarning: LinearSearchBlendedUniformNoiseAttack failed to draw sufficient random inputs that are adversarial (3 / 5).
  warnings.warn(


ValueError: init_attack failed for 2.0 of 5 inputs